# EarthSearch: query and ingest

https://stacindex.org/catalogs/earth-search#/

https://registry.opendata.aws/sentinel-2/

https://viewer.aws.element84.com/

In [1]:
import os
os.environ["abcli_path_bash"] = "{}/git/awesome-bash-cli/bash".format(os.getenv("HOME"))

In [2]:
import boto3
from pystac_client import Client
import datetime
from abcli import fullname
from abcli import file, path, string
from abcli.modules import objects
from blue_geo import env
from blue_geo.catalog.EarthSearch import EarthSearchCatalog, EarthSearchSentinel2L1CDatacube
from blue_geo import NAME, VERSION
from blue_geo.logger import logger

logger.info(f"{NAME}-{VERSION}.{fullname()}, built on {string.pretty_date()}")

🌐  blue_geo-4.393.1.abcli-9.267.1, built on 02 September 2024, 14:09:31


In [3]:
catalog = EarthSearchCatalog()
catalog.get_list_of_collections()

['cop-dem-glo-30',
 'cop-dem-glo-90',
 'landsat-c2-l2',
 'naip',
 'sentinel-1-grd',
 'sentinel-2-c1-l2a',
 'sentinel-2-l1c',
 'sentinel-2-l2a',
 'sentinel-2-pre-c1-l2a']

In [4]:
URL = "https://earth-search.aws.element84.com/v1/"
client = Client.open(URL)

---

## query

In [5]:
object_name = f"query-{string.timestamp()}"
object_path = objects.object_path(object_name)

logger.info(f"📂 {object_name}")

🌐  📂 query-2024-09-02-14-09-32-84111


In [6]:
query_datatime = datetime.datetime(2024, 7, 30).isoformat() + "/" + datetime.datetime(2024, 8, 9).isoformat()
logger.info(f"query_datatime: {query_datatime}")

🌐  query_datatime: 2024-07-30T00:00:00/2024-08-09T00:00:00


In [7]:
search_parameters = {
    "collections": ["sentinel-2-l1c"],
    "bbox": [-122.78 - 0.1, 51.83 - 0.1, -122.78 + 0.1, 51.83 + 0.1],
    "datetime": query_datatime,
    "limit": 10,
}

In [8]:
search = client.search(**search_parameters)

items = list(search.item_collection())

logger.info(f"{len(items)} item(s) found.")

for item in items:
    logger.info("🧊 {}: {} @ {}".format(item.id, item.datetime, ", ".join(list(item.assets.keys()))))

🌐  8 item(s) found.
🌐  🧊 S2A_10UDC_20240807_0_L1C: 2024-08-07 19:20:16.111000+00:00 @ blue, cirrus, coastal, granule_metadata, green, nir, nir08, nir09, red, rededge1, rededge2, rededge3, swir16, swir22, thumbnail, tileinfo_metadata, visual
🌐  🧊 S2A_10UEC_20240807_0_L1C: 2024-08-07 19:20:12.622000+00:00 @ blue, cirrus, coastal, granule_metadata, green, nir, nir08, nir09, red, rededge1, rededge2, rededge3, swir16, swir22, thumbnail, tileinfo_metadata, visual
🌐  🧊 S2B_10UDC_20240805_0_L1C: 2024-08-05 19:30:15.320000+00:00 @ blue, cirrus, coastal, granule_metadata, green, nir, nir08, nir09, red, rededge1, rededge2, rededge3, swir16, swir22, thumbnail, tileinfo_metadata, visual
🌐  🧊 S2B_10UEC_20240805_0_L1C: 2024-08-05 19:30:10.516000+00:00 @ blue, cirrus, coastal, granule_metadata, green, nir, nir08, nir09, red, rededge1, rededge2, rededge3, swir16, swir22, thumbnail, tileinfo_metadata, visual
🌐  🧊 S2B_10UDC_20240802_0_L1C: 2024-08-02 19:20:16.034000+00:00 @ blue, cirrus, coastal, granule

In [9]:
datacube_id = items[0].id
logger.info(f"🧊 {datacube_id}")

🌐  🧊 S2A_10UDC_20240807_0_L1C


## ingest

In [10]:
search_parameters = {
    "ids": [datacube_id]
}

In [11]:
search = client.search(**search_parameters)

items = list(search.item_collection())

logger.info(f"{len(items)} item(s) found.")

for item in items:
    logger.info("🧊 {}: {} @ {}".format(item.id, item.datetime, ", ".join(list(item.assets.keys()))))

item = items[0]

🌐  1 item(s) found.
🌐  🧊 S2A_10UDC_20240807_0_L1C: 2024-08-07 19:20:16.111000+00:00 @ blue, cirrus, coastal, granule_metadata, green, nir, nir08, nir09, red, rededge1, rededge2, rededge3, swir16, swir22, thumbnail, tileinfo_metadata, visual


In [12]:
item.assets

{'blue': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B02.jp2>,
 'cirrus': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B10.jp2>,
 'coastal': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B01.jp2>,
 'granule_metadata': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/metadata.xml>,
 'green': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B03.jp2>,
 'nir': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B08.jp2>,
 'nir08': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B8A.jp2>,
 'nir09': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B09.jp2>,
 'red': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B04.jp2>,
 'rededge1': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B05.jp2>,
 'rededge2': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B06.jp2>,
 'rededge3': <Asset href=s3://sentinel-s2-l1c/tiles/10/U/DC/2024/8/7/0/B07.jp2>,
 'swir16': <Asset href=s3://sentinel-s2-l

---

In [13]:
# END